In [ ]:
# | default_exp utils.response

# Response utils

> Utilitary functions to displat response

## Imports

Python modules

In [ ]:
# | export
import json
from typing import Any, Dict, Union

Third-party modules

In [ ]:
# | export
from requests import Response

`newrelic_sb_sdk` module

In [ ]:
# | export
from newrelic_sb_sdk.graphql.objects import Account
from newrelic_sb_sdk.utils.exceptions import NewRelicError

## Functions

In [ ]:
# | export


def print_response(response, compact: bool = False):
    """Print response in json format."""
    print(
        json.dumps(
            response.json(),
            indent=None if compact else 4,
        )
    )

In [ ]:
# | export


def get_response_data(
    response, key_path: Union[str, None] = None, action: str = "actor"
) -> Union[Dict[str, Any], None]:
    """Get response body entries from a keypath."""
    data = response.json().get("data").get(action)

    if key_path is not None:
        for key in key_path.split(":"):
            if key.isdecimal() and isinstance(data, list):
                data = data[int(key)]
            else:
                data = data.get(key)

    return data

In [ ]:
# | export


def raise_response_errors(*, response: Response, account: Union[Account, None] = None):
    response.raise_for_status()

    response_json = response.json()

    if errors := response_json.get("errors", None):
        for error in errors:
            message = error["message"]

            if account:
                message = f"{account.id} - {account.name} - {message}"

            raise NewRelicError(message)